In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import boto3
import re
from datetime import datetime

import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig
from sagemaker.model_monitor import DataCaptureConfig, DatasetFormat, DefaultModelMonitor
from sagemaker.s3 import S3Uploader, S3Downloader

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

%cd /root/predicting-coronavirus
#%pip install mpu
import src.data_import as di
import src.data_tools as dt

%matplotlib inline
pd.set_option('display.max_columns', 500)  
pd.set_option('display.max_rows', 500)   

/root/predicting-coronavirus
Note: you may need to restart the kernel to use updated packages.


In [94]:
sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()

In [96]:
account_id = sess.client('sts', region_name=sess.region_name).get_caller_identity()["Account"]
bucket = 'sagemaker-studio-{}-{}'.format(sess.region_name, account_id)
prefix = 'capstone2'

try:
    if sess.region_name == "us-east-1":
        sess.client('s3').create_bucket(Bucket=bucket)
    else:
        sess.client('s3').create_bucket(Bucket=bucket, 
                                        CreateBucketConfiguration={'LocationConstraint': sess.region_name})
except Exception as e:
    print("Looks like you already have a bucket of this name. That's good. Uploading the data files...")

# Return the URLs of the uploaded file, so they can be reviewed or used elsewhere
s3url = S3Uploader.upload('data/train.csv', 's3://{}/{}/{}'.format(bucket, prefix,'merged'))
s3url = S3Uploader.upload('data/val.csv', 's3://{}/{}/{}'.format(bucket, prefix,'merged'))
print(s3url)


s3://sagemaker-studio-us-east-1-752222400982/capstone2/merged/val.csv


In [98]:
from sagemaker.amazon.amazon_estimator import get_image_uri
docker_image_name = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='1.0-1')

In [99]:
s3_input_train = sagemaker.s3_input(s3_data='s3://sagemaker-studio-us-east-1-752222400982/capstone2/merged/train.csv', content_type='csv')
s3_input_val = sagemaker.s3_input(s3_data='s3://sagemaker-studio-us-east-1-752222400982/capstone2/merged/val.csv', content_type='csv')


In [100]:
smsess = sagemaker.session.Session()

create_date = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
coronavirus_xgb_experiment = Experiment.create(experiment_name="predicting-coronavirus-cases-xgboost-{}".format(create_date), 
                                              description="Using xgboost to predict coronavirus cases", 
                                              sagemaker_boto_client=boto3.client('sagemaker'))

In [101]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

trial = Trial.create(trial_name="algorithm-mode-trial-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime())), 
                     experiment_name=coronavirus_xgb_experiment.experiment_name,
                     sagemaker_boto_client=boto3.client('sagemaker'))

xgb = sagemaker.estimator.Estimator(image_name=docker_image_name,
                                    role=role,
                                    train_instance_count=1, 
                                    train_use_spot_instances=True,
                                    train_max_run=300,
                                    train_max_wait=600,
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    base_job_name="xgboost-coronavirus",
                                    sagemaker_session=smsess)

xgb.set_hyperparameters(num_round=30, 
                        objective='reg:squarederror',
                        verbosity=3,
#                        eta=0.1,
                        gamma=4)

hp_ranges = {
    "max_depth":IntegerParameter(1,10),
    "subsample":ContinuousParameter(0.3,0.8),
    "eta":ContinuousParameter(0.03,0.2),
    "min_child_weight":IntegerParameter(2,10),
    "alpha":ContinuousParameter(0, 2)
}

tuner = HyperparameterTuner(
    xgb,
    'validation:rmse',
    hp_ranges,
    objective_type='Minimize',
    max_jobs=200,
    max_parallel_jobs=10
)

tuner.fit({'train': s3_input_train,
         'validation': s3_input_val}, 
                experiment_config={
                    "ExperimentName": coronavirus_xgb_experiment.experiment_name, 
                    "TrialName": trial.trial_name,
                    "TrialComponentDisplayName": "HPO Job"
                })



INFO:root:_TuningJob.start_new!!!
INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-xgboost-200513-2222


In [108]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

exp = HyperparameterTuningJobAnalytics(
    sagemaker_session=smsess,
    hyperparameter_tuning_job_name=tuner.latest_tuning_job.name)


In [109]:
df = exp.dataframe()
df

,alpha,eta,max_depth,min_child_weight,subsample,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,1.570509,0.075379,3.0,10.0,0.302129,sagemaker-xgboost-200513-2222-166-ff92aa25,InProgress,NaN,NaT,NaT,NaN
1,1.096632,0.065311,10.0,10.0,0.312322,sagemaker-xgboost-200513-2222-165-a1749e16,InProgress,NaN,NaT,NaT,NaN
2,0.784254,0.073103,6.0,8.0,0.316005,sagemaker-xgboost-200513-2222-164-6501829c,InProgress,NaN,NaT,NaT,NaN
3,0.657157,0.057921,7.0,9.0,0.314494,sagemaker-xgboost-200513-2222-163-2cbdc5aa,InProgress,NaN,NaT,NaT,NaN
4,0.657157,0.057921,7.0,9.0,0.304494,sagemaker-xgboost-200513-2222-162-a9ad3b3d,InProgress,NaN,NaT,NaT,NaN
5,1.592787,0.069736,10.0,10.0,0.308187,sagemaker-xgboost-200513-2222-161-927219ba,InProgress,NaN,NaT,NaT,NaN
6,0.793410,0.075479,3.0,10.0,0.320747,sagemaker-xgboost-200513-2222-160-dede7bf6,InProgress,NaN,2020-05-13 23:23:31+00:00,NaT,NaN
7,0.290622,0.061359,10.0,7.0,0.302460,sagemaker-xgboost-200513-2222-159-1f638ee2,InProgress,NaN,2020-05-13 23:23:35+00:00,NaT,NaN
8,1.216220,0.061613,9.0,9.0,0.303635,sagemaker-xgboost-200513-2222-158-09bfe774,InProgress,NaN,2020-05-13 23:23:38+00:00,NaT,NaN
9,1.830763,0.066286,6.0,10.0,0.311285,sagemaker-xgboost-200513-2222-157-10522ac6,InProgress,NaN,2020-05-13 23:23:43+00:00,NaT,NaN


In [ ]:
df.sort_values('FinalObjectiveValue', ascending=False)